## Imports

In [61]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time

from scipy.stats import spearmanr
from talib import RSI, BBANDS, MACD, ATR

In [62]:
# # Top 50 companies by market cap
# tickers = [
#     # Technology
#     'AAPL', 'MSFT', 'NVDA', 'GOOGL', 'GOOG', 'AMZN', 'META', 'TSLA', 'AVGO', 'ORCL',
#     'CRM', 'ADBE', 'NFLX', 'AMD', 'INTC', 'CSCO', 'QCOM', 'TXN', 'INTU', 'IBM',
    
#     # Financial Services
#     'BRK-B', 'JPM', 'V', 'MA', 'BAC', 'WFC', 'GS', 'MS', 'C', 'AXP',
    
#     # Healthcare & Pharmaceuticals
#     'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'TMO', 'ABT', 'DHR', 'BMY', 'LLY',
    
#     # Consumer & Retail
#     'COST', 'PG', 'KO', 'PEP', 'WMT', 'HD', 'MCD', 'DIS', 'NKE', 'SBUX'
# ]
tickers = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'GOOG', 'AMZN', 'META', 'TSLA', 'AVGO']

# Clean tickers for yfinance compatibility
tickers = [ticker.replace('.', '-') for ticker in tickers]
print(f"Total tickers: {len(tickers)}")
print(tickers)

Total tickers: 9
['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'GOOG', 'AMZN', 'META', 'TSLA', 'AVGO']


In [63]:
start_date = '2015-01-01'
end_date = '2025-01-01'

The following method is supposed to download data for each ticket.

In [64]:
def get_data(ticker, start_date, end_date):
    data = yf.download(ticker, start_date, end_date)
    data["Ticker"] = ticker
    return data

In [65]:
os.makedirs("data", exist_ok=True)

print(f"Processing data for {len(tickers)} stocks...")
for ticker in tickers:
    try:
        # Get enhanced data for this stock
        stock_data = get_data(ticker, start_date, end_date)
        
        # Reset index to make Date a column
        stock_data = stock_data.reset_index()

        print(f"Processed {ticker} data with {len(stock_data)} rows.")
        
        # Save individual stock data
        stock_data.to_csv(f"data/{ticker}.csv", index=False)
        
        # Add a small delay to avoid hitting API limits
        time.sleep(2)

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")

/tmp/ipykernel_21078/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed

Processing data for 9 stocks...
Processed AAPL data with 2516 rows.



/tmp/ipykernel_21078/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed MSFT data with 2516 rows.


/tmp/ipykernel_21078/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


...

Processed META data with 2516 rows.


/tmp/ipykernel_21078/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed TSLA data with 2516 rows.


/tmp/ipykernel_21078/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed AVGO data with 2516 rows.


Let us analyze how does the dataframe for one stock look like.

In [66]:
files = os.listdir('data') # List of files in the data directory
files

['MSFT.csv',
 'AVGO.csv',
 'AMZN.csv',
 'GOOGL.csv',
 'NVDA.csv',
 'META.csv',
 'TSLA.csv',
 'GOOG.csv',
 'AAPL.csv']

In [67]:
example_file = 'MSFT.csv'
example_file_path = os.path.join('data', example_file)

In [68]:
msft_data = pd.read_csv(example_file_path)
msft_data.head()

,Date,Close,High,Low,Open,Volume,Ticker
0,NaN,MSFT,MSFT,MSFT,MSFT,MSFT,NaN
1,2015-01-02,39.93305969238281,40.49669986025541,39.74518180826074,39.84766095030489,27913900,MSFT
2,2015-01-05,39.56582260131836,39.90742065562575,39.497501035805946,39.59998012632301,39673900,MSFT
3,2015-01-06,38.98511505126953,39.92451407944966,38.891174496900994,39.608534880189445,36447900,MSFT
4,2015-01-07,39.480430603027344,39.676850266793366,38.84847225138925,39.26693054357484,29114100,MSFT


As we see, there is a redundant second row(row with index 0), and moreover there is redundant indexes in the dataframe. We want to get rid of them and concatenate dataframes.  

In [69]:
dataframes = []
for file in files:
    file_path = os.path.join('data', file)     
    stock_data = pd.read_csv(file_path)
    stock_data.drop([0], inplace=True) # Drop row with index 0
    dataframes.append(stock_data)
if dataframes:
    data = pd.concat(dataframes, ignore_index=True)

In [70]:
data

,Date,Close,High,Low,Open,Volume,Ticker
0,2015-01-02,39.93305969238281,40.49669986025541,39.74518180826074,39.84766095030489,27913900,MSFT
1,2015-01-05,39.56582260131836,39.90742065562575,39.497501035805946,39.59998012632301,39673900,MSFT
2,2015-01-06,38.98511505126953,39.92451407944966,38.891174496900994,39.608534880189445,36447900,MSFT
3,2015-01-07,39.480430603027344,39.676850266793366,38.84847225138925,39.26693054357484,29114100,MSFT
4,2015-01-08,40.64186096191406,40.7785008343999,39.898881858881815,39.92450081692555,29645200,MSFT
...,...,...,...,...,...,...,...
22639,2024-12-24,257.2866516113281,257.2965955586333,254.38692663500538,254.58623131585566,23234700,AAPL
22640,2024-12-26,258.1037292480469,259.1799258751944,256.7186620602343,257.27667867815336,27237100,AAPL
22641,2024-12-27,254.6858673095703,257.78488174803636,252.16481819720389,256.91793375888693,42355300,AAPL
22642,2024-12-30,251.307861328125,252.60326573181473,249.86299361835324,251.33775398934142,35557500,AAPL


In [71]:
data.dtypes

Date      object
Close     object
High      object
Low       object
Open      object
Volume    object
Ticker    object
dtype: object

The columns should be converted into appropriate datatype. 

In [72]:
data['Date'] = pd.to_datetime(data['Date'])
numeric_columns = ['Close', 'High', 'Low', 'Open', 'Volume']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [73]:
data.dtypes

Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
Ticker            object
dtype: object

In [74]:
data.columns = data.columns.map(lambda x: x.lower())

#### Compute Rolling Average Dollar Volume

In [75]:
data['dollar_vol'] = data[['close', 'volume']].prod(axis=1)

In [76]:
data

,date,close,high,low,open,volume,ticker,dollar_vol
0,2015-01-02,39.933060,40.496700,39.745182,39.847661,27913900,MSFT,1.114687e+09
1,2015-01-05,39.565823,39.907421,39.497501,39.599980,39673900,MSFT,1.569730e+09
2,2015-01-06,38.985115,39.924514,38.891174,39.608535,36447900,MSFT,1.420926e+09
3,2015-01-07,39.480431,39.676850,38.848472,39.266931,29114100,MSFT,1.149437e+09
4,2015-01-08,40.641861,40.778501,39.898882,39.924501,29645200,MSFT,1.204836e+09
...,...,...,...,...,...,...,...,...
22639,2024-12-24,257.286652,257.296596,254.386927,254.586231,23234700,AAPL,5.977978e+09
22640,2024-12-26,258.103729,259.179926,256.718662,257.276679,27237100,AAPL,7.029997e+09
22641,2024-12-27,254.685867,257.784882,252.164818,256.917934,42355300,AAPL,1.078730e+10
22642,2024-12-30,251.307861,252.603266,249.862994,251.337754,35557500,AAPL,8.935879e+09


In [77]:
data['dollar_vol_1m'] = (data.groupby('ticker')['dollar_vol']
                           .rolling(window=21)
                           .mean()).values

In [78]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22644 entries, 0 to 22643
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           22644 non-null  datetime64[ns]
 1   close          22644 non-null  float64       
 2   high           22644 non-null  float64       
 3   low            22644 non-null  float64       
 4   open           22644 non-null  float64       
 5   volume         22644 non-null  int64         
 6   ticker         22644 non-null  object        
 7   dollar_vol     22644 non-null  float64       
 8   dollar_vol_1m  22464 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 1.6+ MB


In [79]:
data['dollar_vol_rank'] = (data.groupby('date')
                             .dollar_vol_1m
                             .rank(ascending=False))

In [80]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22644 entries, 0 to 22643
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             22644 non-null  datetime64[ns]
 1   close            22644 non-null  float64       
 2   high             22644 non-null  float64       
 3   low              22644 non-null  float64       
 4   open             22644 non-null  float64       
 5   volume           22644 non-null  int64         
 6   ticker           22644 non-null  object        
 7   dollar_vol       22644 non-null  float64       
 8   dollar_vol_1m    22464 non-null  float64       
 9   dollar_vol_rank  22464 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 1.7+ MB


### Add some Basic Factors

### Compute the Relative Strength Index

In [81]:
data['rsi'] = data.groupby('ticker')['close'].transform(RSI)

### Compute Bollinger Bands

In [82]:
def compute_bb(close_prices):
    """
    Compute Bollinger Bands for a single ticker's close prices
    Returns a DataFrame with bb_high and bb_low columns
    """
    high, mid, low = BBANDS(close_prices, timeperiod=20)
    return pd.DataFrame({
        'bb_high': high, 
        'bb_low': low
    }, index=close_prices.index)

In [83]:
bb_results = data.groupby('ticker')['close'].apply(compute_bb)
bb_results = bb_results.reset_index(level=0, drop=True) # Reset index to align with original data
data = data.join(bb_results)

In [84]:
data['bb_high'] = data['bb_high'].sub(data['close']).div(data['bb_high']).apply(np.log1p)
data['bb_low'] = data['close'].sub(data['bb_low']).div(data['close']).apply(np.log1p)

### Compute Average True Range

In [85]:
def compute_atr(stock_data):
    """
    Compute normalized ATR for a single ticker
    """
    try:
        df = ATR(stock_data['high'], stock_data['low'], 
                 stock_data['close'], timeperiod=14)
        return df.sub(df.mean()).div(df.std())
    except:
        return pd.Series(np.nan, index=stock_data.index)


In [86]:
data['atr'] = data.groupby('ticker', group_keys=False).apply(compute_atr, include_groups=False)

### Compute Moving Average
### Convergance/Divergance

In [87]:
def compute_macd(close_prices):
    """
    Compute normalized MACD for a single ticker
    """
    try:
        macd = MACD(close_prices)[0]  # Get only the MACD line
        return (macd - np.mean(macd)) / np.std(macd)
    except:
        return pd.Series(np.nan, index=close_prices.index)


In [88]:
data['macd'] = data.groupby('ticker')['close'].apply(compute_macd).reset_index(level=0, drop=True)

### Compute Lagged Returns

In [89]:
lags = [1, 5, 10, 21, 42, 63]

In [90]:
returns = data.groupby('ticker')['close'].pct_change()
percentiles=[.0001, .001, .01]
percentiles+= [1-p for p in percentiles]
returns.describe(percentiles=percentiles).iloc[2:].to_frame('percentiles').style.format(lambda x: f'{x:,.2%}')

,percentiles
std,2.36%
min,-26.39%
0.01%,-20.76%
0.1%,-12.28%
1%,-6.31%
50%,0.12%
99%,7.12%
99.9%,14.92%
99.99%,24.08%
max,29.81%


In [91]:
q = 0.0001

### Winsorize outliers

In [92]:
for lag in lags:
    data[f'return_{lag}d'] = (data.groupby('ticker')['close']
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(q),
                                                       upper=x.quantile(1 - q)))
                                .add(1)
                                .pow(1 / lag)
                                .sub(1)
                                )

### Shift lagged outliers

In [93]:
for t in [1, 2, 3, 4, 5]:
    for lag in [1, 5, 10, 21]:
        data[f'lag_return_{lag}d'] = (data.groupby('ticker')
                                           [f'return_{lag}d'].shift(t * lag))

### Compute forward returns

In [94]:
for t in [1, 5, 10, 21]:
    data[f'target_{t}d'] = data.groupby('ticker')[f'return_{t}d'].shift(-t)

### Create year and month columns

In [95]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [96]:
data = pd.get_dummies(data,
                      columns=['year', 'month'],
                      prefix=['year', 'month'],
                      prefix_sep=['_', '_'],
                      drop_first=True)

In [97]:
data

,date,close,high,low,open,volume,ticker,dollar_vol,dollar_vol_1m,dollar_vol_rank,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2015-01-02,39.933060,40.496700,39.745182,39.847661,27913900,MSFT,1.114687e+09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2015-01-05,39.565823,39.907421,39.497501,39.599980,39673900,MSFT,1.569730e+09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2015-01-06,38.985115,39.924514,38.891174,39.608535,36447900,MSFT,1.420926e+09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,2015-01-07,39.480431,39.676850,38.848472,39.266931,29114100,MSFT,1.149437e+09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,2015-01-08,40.641861,40.778501,39.898882,39.924501,29645200,MSFT,1.204836e+09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22639,2024-12-24,257.286652,257.296596,254.386927,254.586231,23234700,AAPL,5.977978e+09,3.610322e+10,1.0,...,False,False,False,False,False,False,False,False,False,True
22640,2024-12-26,258.103729,259.179926,256.718662,257.276679,27237100,AAPL,7.029997e+09,3.620858e+10,1.0,...,False,False,False,False,False,False,False,False,False,True
22641,2024-12-27,254.685867,257.784882,252.164818,256.917934,42355300,AAPL,1.078730e+10,3.690446e+10,1.0,...,False,False,False,False,False,False,False,False,False,True
22642,2024-12-30,251.307861,252.603266,249.862994,251.337754,35557500,AAPL,8.935879e+09,3.727750e+10,1.0,...,False,False,False,False,False,False,False,False,False,True


In [98]:
data.shape

(22644, 49)

In [99]:
data.isna().sum()

date                  0
close                 0
high                  0
low                   0
open                  0
volume                0
ticker                0
dollar_vol            0
dollar_vol_1m       180
dollar_vol_rank     180
rsi                 126
bb_high             171
bb_low              171
atr                 126
macd                297
return_1d             9
return_5d            45
return_10d           90
return_21d          189
return_42d          378
return_63d          567
lag_return_1d        54
lag_return_5d       270
lag_return_10d      540
lag_return_21d     1134
target_1d             9
target_5d            45
target_10d           90
target_21d          189
year_2016             0
year_2017             0
year_2018             0
year_2019             0
year_2020             0
year_2021             0
year_2022             0
year_2023             0
year_2024             0
month_2               0
month_3               0
month_4               0
month_5         

In [100]:
data.dropna(inplace=True)

In [101]:
data.isna().sum()

date               0
close              0
high               0
low                0
open               0
volume             0
ticker             0
dollar_vol         0
dollar_vol_1m      0
dollar_vol_rank    0
rsi                0
bb_high            0
bb_low             0
atr                0
macd               0
return_1d          0
return_5d          0
return_10d         0
return_21d         0
return_42d         0
return_63d         0
lag_return_1d      0
lag_return_5d      0
lag_return_10d     0
lag_return_21d     0
target_1d          0
target_5d          0
target_10d         0
target_21d         0
year_2016          0
year_2017          0
year_2018          0
year_2019          0
year_2020          0
year_2021          0
year_2022          0
year_2023          0
year_2024          0
month_2            0
month_3            0
month_4            0
month_5            0
month_6            0
month_7            0
month_8            0
month_9            0
month_10           0
month_11     

In [102]:
data.to_csv('data/stocks.csv', index=False)